In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import albumentations as A
from pathlib import Path
import os
from tqdm import tqdm
import sklearn.metrics
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
from matplotlib.font_manager import FontProperties
prop = FontProperties()
plt.style.use('seaborn-dark-palette')
prop.set_file('../data/kalpurush.ttf')
%matplotlib inline

In [2]:
#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from IPython.display import SVG
from tensorflow.keras import backend as K
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D, AvgPool2D

In [4]:
train_df = pd.read_csv('/home/sayan/Documents/Bengali_Grapheme/data/train.csv')
strs = train_df['image_id'].values
strs = [string+'_f.png' for string in strs]
train_df.drop(['image_id'],axis=1,inplace=True)
train_df['image_id'] = strs

In [5]:
train_df.head()

,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme,image_id
0,15,9,5,ক্ট্রো,Train_0_f.png
1,159,0,0,হ,Train_1_f.png
2,22,3,5,খ্রী,Train_2_f.png
3,53,2,2,র্টি,Train_3_f.png
4,71,9,5,থ্রো,Train_4_f.png


In [6]:
class_map = pd.read_csv('/home/sayan/Documents/Bengali_Grapheme/data/class_map.csv')

In [7]:
def apply_augmentation(image):
    augmentation_pipeline = A.Compose(
        [
            A.OneOf(
                [
                    # apply one of transforms to 50% of images
                    A.RandomContrast(), # apply random contrast
                    A.RandomGamma(), # apply random gamma
                    A.RandomBrightness(), # apply random brightness
                ],
                p = 0.5
            ),
            A.OneOf(
                [
                    # apply one of transforms to 50% images
                    A.ElasticTransform(alpha = 120,sigma = 120 * 0.05,alpha_affine = 120 * 0.03),
                    A.GridDistortion(),
                    A.OpticalDistortion(distort_limit = 2,shift_limit = 0.5),
                    A.ShiftScaleRotate()
                ],
                p = 0.5
            ),
            A.OneOf(
                [
                    # apply one of transforms to 50% images
                    A.GaussianBlur(alpha = 120,sigma = 120 * 0.05,alpha_affine = 120 * 0.03),
                    A.GridDistortion(),
                    A.OpticalDistortion(distort_limit = 2,shift_limit = 0.5),
                    
                    
                ],
                p = 0.5
            )
        ],
        p = 0.5
    )
    images_aug = augmentation_pipeline(image = image)['image']
    return images_aug

def image_scale(image):
    augmentation_pipeline = A.Resize(224,224,always_apply=True)
    images_aug = augmentation_pipeline(image = image)['image']
    
    return images_aug

In [8]:
def modified_recall(y_true, y_pred):
    
    scores = []
    for component in ['grapheme_root', 'consonant_diacritic', 'vowel_diacritic']:
        y_true_subset = solution[solution[component] == component]['target'].values
        y_pred_subset = submission[submission[component] == component]['target'].values
        scores.append(sklearn.metrics.recall_score(
            y_true_subset, y_pred_subset, average='macro'))
    
    final_score = np.average(scores, weights=[2,1,1])

In [9]:
datadir_toy_processed_images = '/home/sayan/Documents/Bengali_Grapheme/images/toy_processed_images'
train_df_sample = pd.DataFrame()
for (directory, _ , image_names) in os.walk(datadir_toy_processed_images):
        train_df_sample = train_df[train_df['image_id'].isin(image_names)]

In [10]:
train_df_sample.shape

(20084, 5)

### Train Test Split

In [11]:
from sklearn.model_selection import train_test_split

train_df_1, valid_df_1 = train_test_split(train_df_sample, random_state=42,
                                          stratify=train_df_sample[['grapheme_root',
                                                                    'vowel_diacritic',
                                                                    'consonant_diacritic']])

#train_df_1, valid_df_1 = train_test_split(train_df_sample)

In [12]:
valid_df_1.head()

,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme,image_id
105777,96,2,5,প্রি,Train_105777_f.png
106468,123,7,0,রে,Train_106468_f.png
10501,55,1,4,ঠ্যা,Train_10501_f.png
22628,103,7,0,ফে,Train_22628_f.png
128257,153,3,0,স্পী,Train_128257_f.png


In [143]:
def oneHotEncode_outputs(gen_image):
    while True:
        output = next(gen_image)
        img = output[0]
        labels = output[1]
        #grapheme_label = np.zeros(168)
        #vowel_label = np.zeros(11)
        #consonant_label = np.zeros(7)
        #for i in range(3):
        #    if i == 0:
        #        a = labels[i]
        #        b = np.zeros((a.size, 168))
        #        b[np.arange(a.size),a] = 1
        #        labels[i] = a
        #    elif i == 1:
        #        a = labels[i]
        #        b = np.zeros((a.size, 11))
        #        b[np.arange(a.size),a] = 1
        #        labels[i] = a
        #    else:
        #        a = labels[i]
        #        b = np.zeros((a.size, 7))
        #        b[np.arange(a.size),a] = 1
        #        labels[i] = a
        yield (img, [np.array(labels[:,0],dtype=np.uint8), np.array(labels[:,1],dtype=np.uint8), np.array(labels[:,2],dtype=np.uint8)])

In [163]:
def get_image_generator():
    
    train_im_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255)
    valid_im_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255)
    train_gen_image = train_im_generator.flow_from_dataframe(train_df_1,
                                                   directory='/home/sayan/Documents/Bengali_Grapheme/images/processed_images',
                                                   x_col='image_id',
                                                   y_col=['grapheme_root','vowel_diacritic','consonant_diacritic'],
                                                   class_mode = 'raw',
                                                   target_size = (64,64),
                                                   color_mode = 'grayscale',
                                                   shuffle=False, 
                                                   batch_size = 1,
                                                   seed=42)
    valid_gen_image = valid_im_generator.flow_from_dataframe(valid_df_1,
                                                   directory='/home/sayan/Documents/Bengali_Grapheme/images/processed_images',
                                                   x_col='image_id',
                                                                 target_size = (64,64),
                                                   y_col=['grapheme_root','vowel_diacritic','consonant_diacritic'],
                                                   class_mode = 'raw',
                                                   color_mode = 'grayscale',
                                                   shuffle=False, 
                                                   batch_size = 1,
                                                   seed=42)
        
    #return (train_gen_image, valid_gen_image)
    return (oneHotEncode_outputs(train_gen_image),oneHotEncode_outputs(valid_gen_image))

### Check if generator labels are accurate

In [ ]:
def image_from_char(char):
    HEIGHT = 236
    WIDTH = 236
    image = Image.new('RGB', (WIDTH, HEIGHT))
    draw = ImageDraw.Draw(image)
    myfont = ImageFont.truetype('../data/kalpurush.ttf', 120)
    w, h = draw.textsize(char, font=myfont)
    draw.text(((WIDTH - w) / 2,(HEIGHT - h) / 3), char, font=myfont)

    return image

In [ ]:
def check_generators(gen_image, train_df,fname,class_map=class_map):
    
    output = next(gen_image)
    fig, axs = plt.subplots(gen_image.batch_size,4, figsize=(64, 64))
    gs1 = gridspec.GridSpec(gen_image.batch_size, 4)
    
    gs1.update(wspace=0.025, hspace=0.05)
    plt.subplots_adjust(wspace=-0.8, hspace=0.1)
    images = output[0]
    roots = output[1][0]
    vowels = output[1][1]
    consonants = output[1][2]
    
    for i in range(gen_image.batch_size): 
        image = images[i].reshape(64,64)
        grapheme_root = roots[i]
        vowel_diacritic = vowels[i]
        consonant_diacritic = consonants[i]
        vowel_label = class_map[(class_map['component_type'] == 'vowel_diacritic') 
                            & (class_map['label'] == vowel_diacritic)]['component'].values[0]
        consonant_label = class_map[(class_map['component_type'] == 'consonant_diacritic') 
                            & (class_map['label'] == consonant_diacritic)]['component'].values[0]
        root_label = class_map[(class_map['component_type'] == 'grapheme_root') 
                            & (class_map['label'] == grapheme_root)]['component'].values[0]
        
        axs[i,0].imshow(image,cmap='Greys')
        axs[i,0].axis('off')
        axs[i,0].set_aspect('equal')
        axs[i,1].imshow(image_from_char(root_label), cmap='Greys')
        axs[i,1].axis('off')
        axs[i,1].set_aspect('equal')
        axs[i,2].imshow(image_from_char(vowel_label), cmap='Greys')
        axs[i,2].axis('off')
        axs[i,2].set_aspect('equal')
        axs[i,3].imshow(image_from_char(consonant_label), cmap='Greys')
        axs[i,3].axis('off')
        axs[i,3].set_aspect('equal')
        
    fig.savefig('../results/'+fname+'.png')

In [ ]:
check_generators(train_gen_image,train_df_1,'full_processed_train_images',class_map);

### Creating Callbacks

In [15]:
class CyclicLR(keras.callbacks.Callback):
    
    def __init__(self,base_lr, max_lr, step_size, base_m, max_m, cyclical_momentum):
 
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.base_m = base_m
        self.max_m = max_m
        self.cyclical_momentum = cyclical_momentum
        self.step_size = step_size
        
        self.clr_iterations = 0.
        self.cm_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}
        
    def clr(self):
        
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        
        if cycle == 2:
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)          
            return self.base_lr-(self.base_lr-self.base_lr/100)*np.maximum(0,(1-x))
        
        else:
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0,(1-x))
    
    def cm(self):
        
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        
        if cycle == 2:
            
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1) 
            return self.max_m
        
        else:
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
            return self.max_m - (self.max_m-self.base_m)*np.maximum(0,(1-x))
        
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())
            
        if self.cyclical_momentum == True:
            if self.clr_iterations == 0:
                K.set_value(self.model.optimizer.momentum, self.cm())
            else:
                K.set_value(self.model.optimizer.momentum, self.cm())
            
            
    def on_batch_begin(self, batch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)
        
        if self.cyclical_momentum == True:
            self.history.setdefault('momentum', []).append(K.get_value(self.model.optimizer.momentum))

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())
        
        if self.cyclical_momentum == True:
            K.set_value(self.model.optimizer.momentum, self.cm())

In [14]:
def get_callbacks(chkpoint_name, log_name, lr_monitor):
    
    checkpoint_cb = keras.callbacks.ModelCheckpoint('../checkpoints/'+chkpoint_name,save_best_only=True)
    
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)
    reduceLR_cb = keras.callbacks.ReduceLROnPlateau(monitor=lr_monitor, factor=0.2,
                                                    patience=5, min_lr=0.001)
    root_logdir = '/home/sayan/Documents/Bengali_Grapheme/logs/'
    
    batch_size = train_gen_image.batch_size
    epochs = 10
    max_lr = 0.5
    base_lr = max_lr/10
    max_m = 0.98
    base_m = 0.85

    cyclical_momentum = True
    augment = True
    cycles = 2.35
    
    def get_run_logdir(name):
        import time
        run_id = time.strftime(name)
        return os.path.join(root_logdir,run_id)

    run_logdir = get_run_logdir(log_name+'-run_%Y_%m_%d-%H_%M_%S')

    tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

    iterations = round(len(train_df)/batch_size*epochs)
    iterations = list(range(0,iterations+1))
    step_size = len(iterations)/(cycles)
    cyclicLR_cb =  CyclicLR(base_lr=base_lr,
                            max_lr=max_lr,
                            step_size=step_size,
                            max_m=max_m,
                            base_m=base_m,
                            cyclical_momentum=cyclical_momentum)
    
    return [checkpoint_cb, reduceLR_cb, tensorboard_cb, early_stopping_cb]

---

### Vanilla Model

In [145]:
def model_vanilla():
    
    input_ = Input(shape=(64,64,1))
    conv1 = Conv2D(filters=6, kernel_size=(3, 3), activation='relu')(input_)
    avgpool1 = AvgPool2D()(conv1)
    
    conv2 = Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(avgpool1)
    avgpool2 = AvgPool2D()(conv2)

    flat = Flatten()(avgpool2)
    
    dense_root_1 = Dense(2000,activation='relu')(flat)
    dense_root_2 = Dense(1500,activation='relu')(dense_root_1)
    dense_root_3 = Dense(1000,activation='relu')(dense_root_2)
    dense_root_4 = Dense(800,activation='relu')(dense_root_3)
    dense_root_5 = Dense(250,activation='relu')(dense_root_4)
    output_root = Dense(168,activation='softmax',name='root')(dense_root_4)
    
    dense_vowel_1 = Dense(800,activation='relu')(flat)
    dense_vowel_2 = Dense(600,activation='relu')(dense_vowel_1)
    dense_vowel_3 = Dense(100,activation='relu')(dense_vowel_2)
    output_vowel = Dense(11,activation='softmax',name='vowel')(dense_vowel_3)
    
    dense_consonant_1 = Dense(800,activation='relu')(flat)
    dense_consonant_2 = Dense(600,activation='relu')(dense_consonant_1)
    dense_consonant_3 = Dense(100,activation='relu')(dense_consonant_2)
    output_consonant = Dense(7,activation='softmax',name='consonant')(dense_consonant_3)
    
    model = keras.Model(inputs=[input_],outputs=[output_root,output_vowel,output_consonant])
    
    return model

In [146]:
keras.backend.clear_session()
model_vanilla = model_vanilla()
keras.utils.plot_model(model_vanilla, '../results/vanilla_model.png', expand_nested=True, show_shapes=True);

In [155]:
model_vanilla.compile(loss = keras.losses.sparse_categorical_crossentropy,
              optimizer = keras.optimizers.Adam(learning_rate=0.05),
              metrics=['accuracy'])

(train_gen_image, valid_gen_image) = get_image_generator(False)

Found 15063 validated image filenames.
Found 5021 validated image filenames.


In [ ]:
history_vanilla = model_vanilla.fit(train_gen_image,
                                    validation_data=valid_gen_image,
                                    epochs = 5)

In [ ]:
model_vanilla.save('../models/vanilla.h5')

---

### Alex Net

In [157]:
def model_alexnet():
    
    input_ = Input(shape=(64,64,1))
    
    conv1 = Conv2D(filters=96, kernel_size=(11, 11), strides=4, padding='valid', activation='relu')(input_)
    maxpool1 = MaxPool2D(pool_size=(3, 3), strides=2, padding='valid')(conv1)
    bn1 = BatchNormalization()(maxpool1)
    
    conv2 = Conv2D(filters=256, kernel_size=(5, 5), strides=1, padding='same', activation='relu')(bn1)
    maxpool2 = MaxPool2D(pool_size=(3, 3), strides=2, padding='valid')(conv2)
    bn2 = BatchNormalization()(maxpool2)
    
    conv3 = Conv2D(filters=384, kernel_size=(3, 3), strides=1, padding='same', activation='relu')(bn2)
    drop1 = Dropout(0.5)(conv3)
    conv4 = Conv2D(filters=384, kernel_size=(3, 3), strides=1, padding='same', activation='relu')(drop1)
    drop2 = Dropout(0.5)(conv4)
    conv5 = Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding='same', activation='relu')(drop2)
    
    flat = Flatten()(conv5)
    
    dense_root_1 = Dense(4096,activation='relu')(flat)
    dense_root_2 = Dense(4096,activation='relu')(dense_root_1)
    dense_root_3 = Dense(1000,activation='relu')(dense_root_2)
    output_root = Dense(168,activation='softmax',name='root')(dense_root_3)
    
    dense_vowel_1 = Dense(800,activation='relu')(flat)
    dense_vowel_2 = Dense(600,activation='relu')(dense_vowel_1)
    dense_vowel_3 = Dense(100,activation='relu')(dense_vowel_2)
    output_vowel = Dense(11,activation='softmax',name='vowel')(dense_vowel_3)
    
    dense_consonant_1 = Dense(800,activation='relu')(flat)
    dense_consonant_2 = Dense(600,activation='relu')(dense_consonant_1)
    dense_consonant_3 = Dense(100,activation='relu')(dense_consonant_2)
    output_consonant = Dense(7,activation='softmax',name='consonant')(dense_consonant_3)
    
    model = keras.Model(inputs=[input_],outputs=[output_root,output_vowel,output_consonant])
    
    return model

In [158]:
keras.backend.clear_session()
model_alexnet = model_alexnet()
keras.utils.plot_model(model_alexnet, '../results/alexnet.png', expand_nested=True, show_shapes=True);

In [159]:
model_alexnet.compile(loss = [keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy],
              optimizer = keras.optimizers.Adam(learning_rate=0.05),
              metrics=['accuracy'])

(train_gen_image, valid_gen_image) = get_image_generator()

Found 15063 validated image filenames.
Found 5021 validated image filenames.


In [161]:
history_alexnet = model_alexnet.fit(train_gen_image,
                                    validation_data=valid_gen_image,
                                    epochs = 5)

Epoch 1/5
    513/Unknown - 57s 111ms/step - loss: 384386.3462 - root_loss: 374217.6562 - vowel_loss: 5204.3413 - consonant_loss: 4961.9897 - root_accuracy: 0.0117 - vowel_accuracy: 0.1602 - consonant_accuracy: 0.6191

KeyboardInterrupt: 

In [ ]:
model_alexnet.save('../models/alexnet.h5')

---

### Kaggle Net

In [ ]:
def bengali_kaggle():
    
    input_ = Input(shape = (64, 64, 1))
#    model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu', input_shape=(137, 236, 1))(input_)
#    model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
#    model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
#    model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
#    model = BatchNormalization(momentum=0.15)(model)
#    model = MaxPool2D(pool_size=(2, 2))(model)
#    model = Conv2D(filters=32, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
#    model = Dropout(rate=0.3)(model)
    
    model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu', input_shape=(64, 64, 1))(input_)
    model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
    model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
    model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
    model = BatchNormalization(momentum=0.15)(model)
    model = MaxPool2D(pool_size=(2, 2))(model)
    model = Conv2D(filters=64, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
    model = BatchNormalization(momentum=0.15)(model)
    model = Dropout(rate=0.3)(model)
    
    model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
    model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
    model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
    model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
    model = BatchNormalization(momentum=0.15)(model)
    model = MaxPool2D(pool_size=(2, 2))(model)
    model = Conv2D(filters=128, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
    model = BatchNormalization(momentum=0.15)(model)
    model = Dropout(rate=0.3)(model)
    
    model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
    model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
    model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
    model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
    model = BatchNormalization(momentum=0.15)(model)
    model = MaxPool2D(pool_size=(2, 2))(model)
    model = Conv2D(filters=256, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
    model = BatchNormalization(momentum=0.15)(model)
    model = Dropout(rate=0.3)(model)
    
    model = Flatten()(model)
    model = Dense(1024, activation = "relu")(model)
    model = Dropout(rate=0.3)(model)
    dense = Dense(512, activation = "relu")(model)
    
    head_root = Dense(168, activation = 'softmax',name='root')(dense)
    head_vowel = Dense(11, activation = 'softmax',name='vowel')(dense)
    head_consonant = Dense(7, activation = 'softmax',name='consonant')(dense)
    
    model = keras.Model(inputs=input_, outputs=[head_root, head_vowel, head_consonant])
    return model

In [ ]:
keras.backend.clear_session()
model_kaggle = bengali_kaggle()
keras.utils.plot_model(model_kaggle, '../results/bengali_kaggle.png', expand_nested=True, show_shapes=True);

In [ ]:
model_kaggle.compile(loss = [keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy],
              optimizer = keras.optimizers.Adam(learning_rate=0.05),
              metrics=['accuracy'])

(train_gen_image, valid_gen_image) = get_image_generator(False)

In [ ]:
history_kaggle = model_kaggle.fit(train_gen_image,
                                    validation_data=valid_gen_image,
                                    epochs = 5,callbacks=get_callbacks('bengali_kaggle.h5',
                                                                       'bengali_kaggle.h5','vowel_loss'))

In [ ]:
model_kaggle.save('../models/bengali_kaggle.h5')

---

### Resnet50

In [ ]:
keras.backend.clear_session()
base_model_resnet = keras.applications.resnet_v2.ResNet152V2(weights='imagenet',
                                                      include_top=False,
                                                      input_shape= (224,224,3))

In [ ]:
model_resnet = keras.layers.GlobalAveragePooling2D()(base_model_resnet.output)
head_root = (Dense(168, activation = 'softmax',name='root'))(model_resnet)
head_vowel = Dense(11, activation = 'softmax',name='vowel')(model_resnet)
head_consonant = Dense(7, activation = 'softmax',name='consonant')(model_resnet)

In [ ]:
model_resnet = keras.Model(inputs=base_model_resnet.input,outputs=[head_root, head_vowel, head_consonant])
keras.utils.plot_model(model_resnet, '../results/resnet.png', expand_nested=True, show_shapes=True);

In [ ]:
for layer in base_model_resnet.layers:
    layer.trainable = False

In [ ]:
for layer in model_resnet.layers:
    print(str(layer)+ ' ' +str(layer.trainable))

In [ ]:
model_resnet.compile(loss = [keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy],
              optimizer = keras.optimizers.Adam(learning_rate=0.05),
              metrics=['accuracy'])

(train_gen_image, valid_gen_image) = get_image_generator(True)

In [ ]:
history_resnet = model_resnet.fit(train_gen_image,
                                  validation_data=valid_gen_image,
                                  epochs = 3,callbacks=get_callbacks('resnet50.h5','resnet50.h5','vowel_loss'))

In [ ]:
model_resnet.save('../models/resnet50.h5')

---

### VGGNet

In [ ]:
keras.backend.clear_session()
base_model_vggnet = keras.applications.vgg16.VGG16(weights='imagenet',
                                                      include_top=False,
                                                      input_shape= (224,224,3))

In [ ]:
model_vggnet = keras.layers.GlobalAveragePooling2D()(base_model_vggnet.output)
head_root = (Dense(168, activation = 'softmax',name='root'))(model_vggnet)
head_vowel = Dense(11, activation = 'softmax',name='vowel')(model_vggnet)
head_consonant = Dense(7, activation = 'softmax',name='consonant')(model_vggnet)

In [ ]:
model_vggnet = keras.Model(inputs=base_model_vggnet.input,outputs=[head_root, head_vowel, head_consonant])
keras.utils.plot_model(model_vggnet, '../results/vggnet.png', expand_nested=True, show_shapes=True);

In [ ]:
for layer in base_model_vggnet.layers:
    layer.trainable = False

In [ ]:
for layer in model_vggnet.layers:
    print(str(layer)+ ' ' +str(layer.trainable))

In [ ]:
model_vggnet.compile(loss = [keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy],
              optimizer = keras.optimizers.Adam(learning_rate=0.05),
              metrics=['accuracy'])

(train_gen_image, valid_gen_image) = get_image_generator(True)

In [ ]:
history_vggnet = model_vggnet.fit(train_gen_image,
                                  validation_data=valid_gen_image,
                                  epochs = 3,callbacks=get_callbacks('vggnet50.h5','vggnet50.h5','vowel_loss'))

In [ ]:
model_vggnet.save('../models/vgg16.h5')

---

### Xception

In [ ]:
keras.backend.clear_session()
base_model_xception = keras.applications.xception.Xception(weights='imagenet',
                                                      include_top=False,
                                                      input_shape= (224,224,3))

In [ ]:
model_xception = keras.layers.GlobalAveragePooling2D()(base_model_xception.output)
head_root = (Dense(168, activation = 'softmax',name='root'))(model_xception)
head_vowel = Dense(11, activation = 'softmax',name='vowel')(model_xception)
head_consonant = Dense(7, activation = 'softmax',name='consonant')(model_xception)

In [ ]:
model_xception = keras.Model(inputs=base_model_xception.input,outputs=[head_root, head_vowel, head_consonant])
keras.utils.plot_model(model_xception, '../results/xception.png', expand_nested=True, show_shapes=True);

In [ ]:
for layer in base_model_xception.layers:
    layer.trainable = False

In [ ]:
model_xception.compile(loss = [keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy],
              optimizer = keras.optimizers.Adam(learning_rate=0.05),
              metrics=['accuracy'])

(train_gen_image, valid_gen_image) = get_image_generator(True)

In [ ]:
history_xception = model_xception.fit(train_gen_image,
                                  validation_data=valid_gen_image,
                                  epochs = 3,callbacks=get_callbacks('xception.h5','xception.h5','vowel_loss'))

In [ ]:
model_xception.save('../models/xception.h5')

---

### Inception

In [ ]:
keras.backend.clear_session()
base_model_inception = keras.applications.inception_v3.InceptionV3(weights='imagenet',
                                                      include_top=False,
                                                      input_shape= (224,224,3))

In [ ]:
model_inception = keras.layers.GlobalAveragePooling2D()(base_model_inception.output)
head_root = (Dense(168, activation = 'softmax',name='root'))(model_inception)
head_vowel = Dense(11, activation = 'softmax',name='vowel')(model_inception)
head_consonant = Dense(7, activation = 'softmax',name='consonant')(model_inception)

In [ ]:
model_inception = keras.Model(inputs=base_model_inception.input,outputs=[head_root, head_vowel, head_consonant])
keras.utils.plot_model(model_inception, '../results/inception.png', expand_nested=False, show_shapes=True);

In [ ]:
for layer in base_model_inception.layers:
    layer.trainable = False

In [ ]:
model_inception.compile(loss = [keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy],
              optimizer = keras.optimizers.Adam(learning_rate=0.05),
              metrics=['accuracy'])

(train_gen_image, valid_gen_image) = get_image_generator(True)

In [ ]:
history_inception = model_inception.fit(train_gen_image,
                                  validation_data=valid_gen_image,
                                  epochs = 3,callbacks=get_callbacks('inception.h5','inception.h5','vowel_loss'))

In [ ]:
model_inception.save('../models/inception.h5')